# Initialize

In [ ]:
# Authors: Martin Luessi <mluessi@nmr.mgh.harvard.edu>
#          Alexandre Gramfort <alexandre.gramfort@inria.fr>
#          Nicolas P. Rougier (graph code borrowed from his matplotlib gallery)
#
# License: BSD (3-clause)

import numpy as np
import os.path as op
import os
import matplotlib.pyplot as plt
from langouEEG import *
from scipy.stats import *

import mne
import pickle
from mne.datasets import sample
from mne.minimum_norm import apply_inverse_epochs, read_inverse_operator
from mne.connectivity import spectral_connectivity, envelope_correlation
from mne.viz import circular_layout, plot_connectivity_circle
import mne
from mne.datasets import eegbci
from mne.datasets import fetch_fsaverage
from mne.datasets import sample
from mne.minimum_norm import make_inverse_operator, apply_inverse
from mne.minimum_norm import write_inverse_operator

import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa
import mne

sample_data_folder = mne.datasets.sample.data_path()
dataRoot = "/data/home/viscent/Light"
# Download fsaverage files
fs_dir = fetch_fsaverage(verbose=True)
isMale = False
isAll = True
isBlind = False
if not isAll:
    result_dir = op.join(dataRoot,'result','male' if isMale else 'female')
else:
    result_dir = op.join(dataRoot,'result','all')
if isBlind:
    result_dir = op.join(result_dir,'Blind')
subjects_dir = op.dirname(fs_dir)
if not op.exists(result_dir):
    os.mkdir(result_dir)
# The files live in:
subject = 'fsaverage'
trans = 'fsaverage'  # MNE has a built-in fsaverage transformation
src = op.join(fs_dir, 'bem', 'fsaverage-ico-5-src.fif')
bem = op.join(fs_dir, 'bem', 'fsaverage-5120-5120-5120-bem-sol.fif')
print(__doc__)

# Load data

In [ ]:
epochs_4F = dict()
epochs_RF = dict()
epochs_4R = dict()
epochs_RR = dict()
for subject_name in range(1,21):
    if not isAll:
        if not (isMale ^ (subject_name in [7,8,11,17])):
            continue
    if subject_name<10:
        subject_name='S0'+str(subject_name)
    else:
        subject_name='S'+str(subject_name)
    with open(dataRoot+'/clean_data_av/'+subject_name+'_clean.lgeeg','rb') as f:
        raw=pickle.load(f)
    events, event_dict=extractEvents(raw)
    picks = mne.pick_types(raw.info, meg=False, eeg=True, stim=False, eog=True,
                        exclude='bads')
    if isBlind:
        epoch_RR,epoch_RF,epoch_4R,epoch_4F = extractEpochsBlind(raw,events,picks)
    else:
        epoch_RR,epoch_RF,epoch_4R,epoch_4F = extractEpochs(raw,events,picks)
    epochs_4F[subject_name]=epoch_4F
    epochs_RF[subject_name]=epoch_RF
    epochs_RR[subject_name]=epoch_RR
    epochs_4R[subject_name]=epoch_4R
    if not op.exists(os.path.join(dataRoot,'fwd_solution.lgeeg')):
        fwd = mne.make_forward_solution(raw.info, trans=trans, src=src,
                                        bem=bem, eeg=True, mindist=5.0, n_jobs=1)
        print(fwd)
        mne.write_forward_solution(os.path.join(dataRoot,'fwd_solution.lgeeg'),fwd,overwrite=True)
    else:
        fwd = mne.read_forward_solution(os.path.join(dataRoot,'fwd_solution.lgeeg'))
    data_path = sample.data_path()

    picks = mne.pick_types(raw.info, meg=False, eeg=True, stim=False, eog=True,
                        exclude='bads')


 




# Source Estimation


In [ ]:
cons_40 = []
cons_rand = []
act_40 = []
act_40_paired = []
act_rand = []
act_rand_paired = []
result_root= result_dir
for subject_name,_ in epochs_4F.items():
    # Compute inverse solution and for each epoch. By using "return_generator=True"
    # stcs will be a generator object instead of a list.
    result_dir = op.join(result_root,subject_name)
    epoch_4F = epochs_4F[subject_name]
    epoch_RF = epochs_RF[subject_name]
    fname_stc_4F = op.join(result_dir,subject_name+'_stc_4F.lgeeg')
    fname_stc_RF = op.join(result_dir,subject_name+'_stc_RF.lgeeg')
    fname_tl_4F = op.join(result_dir,subject_name+'_tl_4F.lgeeg')
    fname_tl_RF = op.join(result_dir,subject_name+'_tl_RF.lgeeg')
    fname_inv = os.path.join(dataRoot,'inv_operators',subject_name+'_inv.lgeeg')
    

    if not os.path.exists(result_dir):
        os.mkdir(result_dir)



    fname_inv_r = os.path.join(dataRoot,'inv_operators',subject_name+'_r_inv.lgeeg')
    fname_cov_r = os.path.join(dataRoot,'noise_covariance',subject_name+'_r_cov.lgeeg')
    fname_inv_4 = os.path.join(dataRoot,'inv_operators',subject_name+'_4_inv.lgeeg')
    fname_cov_4 = os.path.join(dataRoot,'noise_covariance',subject_name+'_4_cov.lgeeg')



    if not os.path.exists(fname_cov_4):  
        noise_cov = mne.compute_covariance(
            epochs_4R[subject_name], tmax=80., method=['shrunk', 'empirical'], rank=None, verbose=True)
        mne.write_cov(fname_cov_4,noise_cov)
    else:   
    # Load data
        noise_cov = mne.read_cov(fname_cov_4)
    
    if not os.path.exists(fname_inv_4):  
        inverse_operator = make_inverse_operator(
            raw.info, fwd, noise_cov, loose=0.2, depth=0.8)
        write_inverse_operator(fname_inv_4,inverse_operator)
    inverse_operator = read_inverse_operator(fname_inv_4)



    snr = 1.0  # use lower SNR for single epochs
    lambda2 = 1.0 / snr ** 2
    method = "dSPM"  # use dSPM method (could also be MNE or sLORETA)
    # Get labels for FreeSurfer 'aparc' cortical parcellation with 34 labels/hemi
    labels = mne.read_labels_from_annot('fsaverage', parc='aparc',
                                        subjects_dir=subjects_dir)[:68]
    label_colors = [label.color for label in labels]
    # Average the source estimates within each label using sign-flips to reduce
    # signal cancellations, also here we return a generator
    src = inverse_operator['src']
#=====STC======

    if op.exists(fname_stc_4F):
        with open(fname_stc_4F,'rb') as f :
            stcs = pickle.load(f)
        print(subject_name+' loaded')
    else:
        stcs = apply_inverse_epochs(epoch_4F, inverse_operator, lambda2, method,
                                    pick_ori="normal", return_generator=False)
        with open(fname_stc_4F,'wb') as f:
            pickle.dump(stcs,f)
#======Time Label=====
    if op.exists(fname_tl_4F):
        with open(fname_tl_4F,'rb') as f:
            label_ts = pickle.load(f)
        print(subject_name+' loaded')

    else:
        
        label_ts = mne.extract_label_time_course(
            stcs, labels,  src,allow_empty=False, mode='mean_flip', return_generator=False)
        with open(fname_tl_4F,'wb') as f:
            pickle.dump(label_ts,f)
    fmin = 8.
    fmax = 13.
    sfreq = raw.info['sfreq']  # the sampling frequency
    con_methods = ['pli', 'wpli2_debiased', 'ciplv']
    if not os.path.exists(op.join(result_dir,'cons')):
        os.mkdir(op.join(result_dir,'cons'))
    if not op.exists(op.join(result_dir,'cons','40_con.lgeeg')):
        # con, freqs, times, n_epochs, n_tapers = spectral_connectivity(
        #     label_ts, method=con_methods, mode='multitaper', sfreq=sfreq, fmin=fmin,
        #     fmax=fmax, faverage=True, mt_adaptive=True, n_jobs=1)
        print("computing correlation")
        con = envelope_correlation(
            label_ts,combine = None,absolute=True,verbose=True)
        with open(op.join(result_dir,'cons','40_con.lgeeg'),'wb') as f:
            pickle.dump(con,f)
    else:
        with open(op.join(result_dir,'cons','40_con.lgeeg'),'rb') as f:
            con = pickle.load(f)
    label_names = [label.name for label in labels]
    # con = np.squeeze(con)
    con = np.mean(con,axis=0)
    cons_40.append(con)
    pd_40=pd.DataFrame(con)
    pd_40.columns = label_names
    pd_40.index = label_names
    pd_40.to_excel(op.join(result_dir,'40_conn.xlsx'))
    con_40 = con.copy()
    pd_40=pd.DataFrame(np.mean(label_ts,axis=2))
    pd_40.columns = label_names
    pd_40.to_excel(op.join(result_dir,'40_activation.xlsx'))
    act_40.append(np.mean(np.array(label_ts),axis=2))
    act_40_paired.append(np.mean(np.mean(np.array(label_ts),axis=2),axis=0))

    if not os.path.exists(fname_cov_r):  
        noise_cov = mne.compute_covariance(
            epochs_RR[subject_name], tmax=80., method=['shrunk', 'empirical'], rank=None, verbose=True)
        mne.write_cov(fname_cov_r,noise_cov)
    else:   
    # Load data
        noise_cov = mne.read_cov(fname_cov_r)
    
    if not os.path.exists(fname_inv_r):  
        inverse_operator = make_inverse_operator(
            raw.info, fwd, noise_cov, loose=0.2, depth=0.8)
        write_inverse_operator(fname_inv_r,inverse_operator)
    inverse_operator = read_inverse_operator(fname_inv_r)


    lambda2 = 1.0 / snr ** 2
    method = "dSPM"  # use dSPM method (could also be MNE or sLORETA)



    #=====STC======

    if op.exists(fname_stc_RF):
        with open(fname_stc_RF,'rb') as f :
            stcs = pickle.load(f)
        print(subject_name+" loaded")
    else:
        stcs = apply_inverse_epochs(epoch_RF, inverse_operator, lambda2, method,
                                    pick_ori="normal", return_generator=False)
        with open(fname_stc_RF,'wb') as f:
            pickle.dump(stcs,f)
#======Time Label=====
    if op.exists(fname_tl_RF):
        with open(fname_tl_RF,'rb') as f:
            label_ts = pickle.load(f)
        print(subject_name+" loaded")
    else:
        label_ts = mne.extract_label_time_course(
            stcs, labels,  src,allow_empty=False, mode='mean_flip', return_generator=False)
        with open(fname_tl_RF,'wb') as f:
            pickle.dump(label_ts,f)

    
    if not op.exists(op.join(result_dir,'cons','rand_con.lgeeg')):
        # con, freqs, times, n_epochs, n_tapers = spectral_connectivity(
        #     label_ts, method=con_methods, mode='multitaper', sfreq=sfreq, fmin=fmin,
        #     fmax=fmax, faverage=True, mt_adaptive=True, n_jobs=1)
        print("computing correlation")
        con = envelope_correlation(
            label_ts,combine = None, absolute=True,verbose=True)
        with open(op.join(result_dir,'cons','rand_con.lgeeg'),'wb') as f:
            pickle.dump(con,f)
    else:
        print(subject_name+" loaded")
        with open(op.join(result_dir,'cons','rand_con.lgeeg'),'rb') as f:
            con = pickle.load(f)
    # con is a 3D array, get the connectivity for the first (and only) freq. band
    # for each method

    # con = np.squeeze(con)
    con = np.mean(con,axis=0)
    cons_rand.append(con)
    con_rand = con.copy()
    pd_rand=pd.DataFrame(con)
    pd_rand.columns = label_names
    pd_rand.index = label_names
    pd_rand.to_excel(op.join(result_dir,'rand_conn.xlsx'))
    pd_delta=pd.DataFrame(con_40-con_rand)
    pd_delta.columns = label_names
    pd_delta.index = label_names
    pd_delta.to_excel(op.join(result_dir,'delta_conn.xlsx'))
    pd_rand=pd.DataFrame(np.mean(label_ts,axis=2))
    pd_rand.columns = label_names
    pd_rand.to_excel(op.join(result_dir,'rand_activation.xlsx'))
    act_rand.append(np.mean(np.array(label_ts),axis=2))
    act_rand_paired.append(np.mean(np.mean(np.array(label_ts),axis=2),axis=0))
cons_40 = np.array(cons_40)
cons_rand = np.array(cons_rand)
act_40 = np.vstack(act_40)
act_rand = np.vstack(act_rand)
act_40_paired = np.array(act_40_paired)
act_rand_paired = np.array(act_rand_paired)

In [ ]:
act_40_norm = act_40.copy()
for i in range(act_40.shape[0]):
    act_40_norm[i] = (act_40[i]-act_40[i].mean())/act_40[i].std()
act_40_lh = act_40_norm[:,::2]
act_40_rh = act_40_norm[:,1::2]
act_40_lh_mean = act_40_lh.mean(axis=1)
act_40_rh_mean = act_40_rh.mean(axis=1)

In [ ]:
cons_40.shape

In [ ]:
plt.plot(act_40_lh_mean-act_40_rh_mean)

In [ ]:
# print('The p-value of L/R difference is: '+str(ttest_rel(act_40_lh_mean,act_40_rh_mean,axis=0).pvalue))
pval = ttest_ind(act_40,act_rand,axis=0).pvalue
plt.plot(np.where(pval<0.05,1,0))
for i in range(68):
    if pval[i] <0.05 :
        print(label_names[i])
# fig, ax = plt.subplots()
# ax.pcolormesh(act_40_lh-act_40_rh)

In [ ]:
pval = 1-ttest_rel(cons_40,cons_rand,axis=0).pvalue
pval_mat=np.where(pval>0.95,pval-0.95,0)
fig, ax = plt.subplots()
ax.pcolormesh(pval_mat)

In [ ]:
pval = 1-ttest_rel(cons_40,cons_rand,axis=0).pvalue
fig, ax = plt.subplots()
pmin = 0.99
ax.pcolormesh(np.where(pval>pmin,pval-pmin,0))
fig, ax = plt.subplots()
ax.pcolormesh(np.where(pval>0.999,pval-pmin,0))

In [ ]:
result_dir = op.join(dataRoot, 'result','overall')
if not op.exists(result_dir):
    os.makedirs(result_dir)
# First, we reorder the labels based on their location in the left hemi
label_names = [label.name for label in labels]

lh_labels = [name for name in label_names if name.endswith('lh')]

# Get the y-location of the label
label_ypos = list()
for name in lh_labels:
    idx = label_names.index(name)
    ypos = np.mean(labels[idx].pos[:, 1])
    label_ypos.append(ypos)

# Reorder the labels based on their location
lh_labels = [label for (yp, label) in sorted(zip(label_ypos, lh_labels))]

# For the right hemi
rh_labels = [label[:-2] + 'rh' for label in lh_labels]

# Save the plot order and create a circular layout
node_order = list()
node_order.extend(lh_labels[::-1])  # reverse the order
node_order.extend(rh_labels)
# node_order = node_order[:69]

node_angles = circular_layout(label_names, node_order, start_pos=90,
                              group_boundaries=[0, len(label_names) / 2])

# Plot the graph using node colors from the FreeSurfer parcellation. We only
# show the 300 strongest connections.
fig,ax=plot_connectivity_circle(cons_40.mean(axis=0), label_names,padding=10, n_lines=300,
                         node_angles=node_angles, node_colors=label_colors, 
                         title='All-to-All Connectivity 40 Hz '
                               'Condition (PLI)')
fig.savefig(op.join(result_dir,'40-connectivity.png'),dpi=300,facecolor='black')
fig,ax=plot_connectivity_circle(cons_rand.mean(axis=0), label_names,padding=10, n_lines=300,
                         node_angles=node_angles, node_colors=label_colors, 
                         title='All-to-All Connectivity Random '
                               'Condition (PLI)')
fig.savefig(op.join(result_dir,'rand-connectivity.png'),dpi=300,facecolor='black')
dcon_res = cons_40.mean(axis=0) - cons_rand.mean(axis=0)
fig,ax=plot_connectivity_circle(np.where(dcon_res<0,0,dcon_res), label_names,padding=10, n_lines=300,
                         node_angles=node_angles, node_colors=label_colors, colormap='hot', 
                         title='All-to-All Connectivity Upregulated '
                               '(PLI)')
fig.savefig(op.join(result_dir,'up-connectivity.png'),dpi=300,facecolor='black')
fig,ax=plot_connectivity_circle(np.where(dcon_res>0,0,-1*dcon_res), label_names,padding=10, n_lines=300,
                         node_angles=node_angles, node_colors=label_colors, colormap='gray',
                         title='All-to-All Connectivity Downregulated '
                               '(PLI)')
fig.savefig(op.join(result_dir,'down-connectivity.png'),dpi=300,facecolor='black')


In [ ]:
pval = 1-ttest_rel(cons_40,cons_rand,axis=0).pvalue
pval_mat = np.where(pval>0.95,1-pval,0)
fig,ax=plot_connectivity_circle(pval_mat, label_names,padding=10, n_lines=300,
                         node_angles=node_angles, node_colors=label_colors, colormap='hot', 
                         title='Robustly Altered Connectivity'
                               '(p<0.05)')
fig.savefig(op.join(result_dir,'0.05.png'),dpi=300,facecolor='black')

pval_mat = np.where(pval>0.99,1-pval,0)
fig,ax=plot_connectivity_circle(pval_mat, label_names,padding=10, n_lines=300,
                         node_angles=node_angles, node_colors=label_colors, colormap='hot', 
                         title='Robustly Altered Connectivity'
                               '(p<0.01)')
fig.savefig(op.join(result_dir,'0.01.png'),dpi=300,facecolor='black')

pval_mat = np.where(pval>0.999,1-pval,0)
fig,ax=plot_connectivity_circle(pval_mat, label_names,padding=10, n_lines=300,
                         node_angles=node_angles, node_colors=label_colors, colormap='hot', 
                         title='Robustly Altered Connectivity'
                               '(p<0.001)')
fig.savefig(op.join(result_dir,'0.001.png'),dpi=300,facecolor='black')